In [ ]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
with open('./Data/emoji_data.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

In [ ]:
data = loaded_data['data']
tokenizer = loaded_data['tokenizer']
emoji_dict = loaded_data['emoji_dict']
embedding_matrix = loaded_data['embedding_matrix']

In [ ]:
X = data['padded_sequences']
y = data['emoji_labels']

X_array = np.array(X.tolist(), dtype=np.int32)
y_array = np.array(y.tolist(), dtype=np.float32)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((X_array, y_array))

BATCH_SIZE = 128
dataset = dataset.shuffle(buffer_size=len(X)).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
del data, loaded_data, X, y

In [ ]:
import gc
gc.collect()

0

In [ ]:
model = Sequential()

embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1,
                            output_dim=300,
                            weights=[embedding_matrix],
                            input_shape=(X_array.shape[1],),
                            trainable=False)
model.add(embedding_layer)
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(emoji_dict), activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 47, 300)             │      42,654,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 47, 128)             │         219,648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 99)                  │          12,771 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 43,018,003 (164.10 MB)

 Trainable params: 364,003 (1.39 MB)

 Non-trainable params: 42,654,000 (162.71 MB)

In [ ]:
history = model.fit(dataset, epochs=15)

Epoch 1/15
5801/5801 ━━━━━━━━━━━━━━━━━━━━ 73s 12ms/step - accuracy: 0.1381 - loss: 3.8720
Epoch 2/15
5801/5801 ━━━━━━━━━━━━━━━━━━━━ 65s 11ms/step - accuracy: 0.2625 - loss: 3.2429
Epoch 3/15
5801/5801 ━━━━━━━━━━━━━━━━━━━━ 64s 11ms/step - accuracy: 0.3280 - loss: 2.9353
Epoch 4/15
5801/5801 ━━━━━━━━━━━━━━━━━━━━ 82s 11ms/step - accuracy: 0.3783 - loss: 2.7116
Epoch 5/15
5801/5801 ━━━━━━━━━━━━━━━━━━━━ 63s 11ms/step - accuracy: 0.4161 - loss: 2.5422
Epoch 6/15
5801/5801 ━━━━━━━━━━━━━━━━━━━━ 83s 11ms/step - accuracy: 0.4447 - loss: 2.4150
Epoch 7/15
5801/5801 ━━━━━━━━━━━━━━━━━━━━ 63s 11ms/step - accuracy: 0.4674 - loss: 2.3128
Epoch 8/15
5801/5801 ━━━━━━━━━━━━━━━━━━━━ 83s 11ms/step - accuracy: 0.4865 - loss: 2.2294
Epoch 9/15
5801/5801 ━━━━━━━━━━━━━━━━━━━━ 63s 11ms/step - accuracy: 0.5008 - loss: 2.1640
Epoch 10/15
5801/5801 ━━━━━━━━━━━━━━━━━━━━ 63s 11ms/step - accuracy: 0.5146 - loss: 2.1064
Epoch 11/15
5801/5801 ━━━━━━━━━━━━━━━━━━━━ 64s 11ms/step - accuracy: 0.5229 - loss: 2.0645
Epoch 12

In [ ]:
model.save("./model/model.h5")